In [1]:
import pymc3 as pm
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import theano.tensor as tt
plt.style.use('seaborn-darkgrid')
np.set_printoptions(precision=2)
pd.set_option('display.precision', 2)

In [2]:
z = np.linspace(-10, 10, 100)
logistic = 1 / (1 + np.exp(-z))
plt.plot(z, logistic)
plt.xlabel('$z$', fontsize=16)
plt.ylabel('$logistic(z)$', fontsize=16)
plt.savefig('img501.png')
plt.close()

In [3]:
iris = sns.load_dataset('iris')
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [4]:
sns.stripplot(x='species', y='sepal_length', data=iris, jitter=False)
plt.savefig('img503.png')
plt.close()

In [5]:
sns.violinplot(x='species', y='sepal_length', data=iris)
plt.savefig('img503-1.png')
plt.close()

/Users/kumada/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/seaborn/categorical.py:598: FutureWarning: remove_na is deprecated and is a private function. Do not use.
  kde_data = remove_na(group_data)
/Users/kumada/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/seaborn/categorical.py:826: FutureWarning: remove_na is deprecated and is a private function. Do not use.
  violin_data = remove_na(group_data)


In [6]:
sns.pairplot(iris, hue='species', diag_kind='kde')
plt.savefig('img504.png')
plt.close()

In [7]:
df = iris.query("species == ('setosa', 'versicolor')")
y_0 = pd.Categorical(df['species']).codes
x_n = 'sepal_length'
x_0 = df[x_n].values

In [8]:
y_0

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int8)

In [9]:
x_0

array([5.1, 4.9, 4.7, 4.6, 5. , 5.4, 4.6, 5. , 4.4, 4.9, 5.4, 4.8, 4.8,
       4.3, 5.8, 5.7, 5.4, 5.1, 5.7, 5.1, 5.4, 5.1, 4.6, 5.1, 4.8, 5. ,
       5. , 5.2, 5.2, 4.7, 4.8, 5.4, 5.2, 5.5, 4.9, 5. , 5.5, 4.9, 4.4,
       5.1, 5. , 4.5, 4.4, 5. , 5.1, 4.8, 5.1, 4.6, 5.3, 5. , 7. , 6.4,
       6.9, 5.5, 6.5, 5.7, 6.3, 4.9, 6.6, 5.2, 5. , 5.9, 6. , 6.1, 5.6,
       6.7, 5.6, 5.8, 6.2, 5.6, 5.9, 6.1, 6.3, 6.1, 6.4, 6.6, 6.8, 6.7,
       6. , 5.7, 5.5, 5.5, 5.8, 6. , 5.4, 6. , 6.7, 6.3, 5.6, 5.5, 5.5,
       6.1, 5.8, 5. , 5.6, 5.7, 5.7, 6.2, 5.1, 5.7])

In [10]:
with pm.Model() as model_0:
    alpha = pm.Normal('alpha', mu=0, sd=10)
    beta = pm.Normal('beta', mu=0, sd=10)
    mu = alpha + pm.math.dot(x_0, beta)
    theta = pm.Deterministic('theta', 1 / (1 + pm.math.exp(-mu)))
    bd = pm.Deterministic('bd', -alpha / beta)
    yl = pm.Bernoulli('yl', p=theta, observed=y_0)
    trace_0 = pm.sample(5000)
    
chain_0 = trace_0[1000:]
varnames = ['alpha', 'beta', 'bd']
pm.traceplot(chain_0, varnames)
plt.savefig('img505.png')
plt.close()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta, alpha]
Sampling 4 chains: 100%|██████████| 22000/22000 [00:15<00:00, 1380.92draws/s]
The number of effective samples is smaller than 25% for some parameters.


In [11]:
pm.summary(chain_0, varnames)

,mean,sd,mc_error,hpd_2.5,hpd_97.5,n_eff,Rhat
alpha,-23.35,4.20,8.23e-02,-32.08,-15.64,2208.26,1.0
beta,4.31,0.78,1.52e-02,2.82,5.86,2208.64,1.0
bd,5.42,0.07,6.66e-04,5.28,5.55,12503.60,1.0


In [12]:
theta = chain_0['theta'].mean(axis=0)
assert((100,) == theta.shape)
idx = np.argsort(x_0)
plt.plot(x_0[idx], theta[idx], color='b', lw=3)
plt.vlines([chain_0['bd'].mean()], ymin=0, ymax=1, color='r')
bd_hpd = pm.hpd(chain_0['bd'])

plt.fill_betweenx([0, 1], bd_hpd[0], bd_hpd[1], color='r', alpha=0.5)
plt.plot(x_0, y_0, 'o', color='k')
theta_hpd = pm.hpd(chain_0['theta'])[idx]
plt.fill_between(x_0[idx], theta_hpd[:,0], theta_hpd[:,1], color='b', alpha=0.5)
plt.xlabel(x_n, fontsize=16)
plt.ylabel(r'$\theta$', rotation=0, fontsize=16)
plt.savefig('img506.png')
plt.close()

In [13]:
def classify(n, threshold):
    n = np.array(n)
    mu = chain_0['alpha'].mean() + chain_0['beta'].mean() * n
    prob = 1 / (1 + np.exp(-mu))
    return prob, prob >= threshold

classify([5, 5.5, 6], 0.5)

(array([0.14, 0.59, 0.93]), array([False,  True,  True]))

In [14]:
df = iris.query("species == ('setosa', 'versicolor')")
y_1 = pd.Categorical(df['species']).codes
x_n = ['sepal_length', 'sepal_width']
x_1 = df[x_n].values

In [15]:
x_1.shape

(100, 2)

In [16]:
y_1.shape

(100,)

In [17]:
y_1

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int8)

In [18]:
y_0

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int8)

In [19]:
with pm.Model() as model_1:
    alpha = pm.Normal('alpha', mu=0, sd=10)
    beta = pm.Normal('beta', mu=0, sd=2, shape=len(x_n))
    mu = alpha + pm.math.dot(x_1, beta)
    theta = 1 / (1 + pm.math.exp(-mu))
    bd = pm.Deterministic('bd', -alpha / beta[1] - beta[0] / beta[1] * x_1[:, 0])
    y1 = pm.Bernoulli('y1', p=theta, observed=y_1)
    trace_1 = pm.sample(5000)

chain_1 = trace_1[1000:]
varnames = ['alpha', 'beta', 'bd']
pm.traceplot(chain_1, varnames)
plt.savefig('img507.png')
plt.close()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta, alpha]
Sampling 4 chains: 100%|██████████| 22000/22000 [00:24<00:00, 906.46draws/s]
The acceptance probability does not match the target. It is 0.8812380807700337, but should be close to 0.8. Try to increase the number of tuning steps.


In [20]:
chain_1['bd'].shape

(16000, 100)

In [21]:
x_1.shape

(100, 2)

In [22]:
idx = np.argsort(x_1[:, 0])
ld = chain_1['bd'].mean(0)[idx]
plt.scatter(x_1[:, 0], x_1[:, 1], c=y_0, cmap='viridis')
plt.plot(x_1[:, 0][idx], ld, color='r')

ld_hpd = pm.hpd(chain_1['bd'])[idx]
plt.fill_between(x_1[:, 0][idx], ld_hpd[:, 0], ld_hpd[:, 1], color='r', alpha=0.5)
plt.xlabel(x_n[0], fontsize=16)
plt.ylabel(x_n[1], fontsize=16)
plt.savefig('img508.png')
plt.close()

In [23]:
corr = iris[iris['species'] != 'virginica'].corr()
mask = np.tri(*corr.shape).T
sns.heatmap(corr.abs(), mask=mask, annot=True, cmap='viridis')
plt.savefig('img509.png')

In [24]:
df = iris.query("species == ('setosa', 'versicolor')")
df = df[45:]
y_3 = pd.Categorical(df['species']).codes
x_n = ['sepal_length', 'sepal_width']
x_3 = df[x_n].values

In [25]:
with pm.Model() as model_1:
    alpha = pm.Normal('alpha', mu=0, sd=10)
    beta = pm.Normal('beta', mu=0, sd=2, shape=len(x_n))
    mu = alpha + pm.math.dot(x_3, beta)
    theta = 1 / (1 + pm.math.exp(-mu))
    bd = pm.Deterministic('bd', -alpha / beta[1] - beta[0] / beta[1] * x_3[:, 0])
    y1 = pm.Bernoulli('y3', p=theta, observed=y_3)
    trace_3 = pm.sample(5000)

chain_3 = trace_3[1000:]
varnames = ['alpha', 'beta', 'bd']
pm.traceplot(chain_3, varnames)
plt.savefig('img507_1.png')
plt.close()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta, alpha]
Sampling 4 chains: 100%|██████████| 22000/22000 [00:20<00:00, 1074.41draws/s]
There were 13 divergences after tuning. Increase `target_accept` or reparameterize.
There were 17 divergences after tuning. Increase `target_accept` or reparameterize.
There were 15 divergences after tuning. Increase `target_accept` or reparameterize.
There were 19 divergences after tuning. Increase `target_accept` or reparameterize.


In [26]:
chain_3['bd'].shape

(16000, 55)

In [27]:
x_3.shape

(55, 2)

In [28]:
idx = np.argsort(x_3[:, 0])
ld = chain_3['bd'].mean(0)[idx]
plt.scatter(x_3[:, 0], x_3[:, 1], c=y_3, cmap='viridis')
plt.plot(x_3[:, 0][idx], ld, color='r')

ld_hpd = pm.hpd(chain_3['bd'])[idx]
plt.fill_between(x_3[:, 0][idx], ld_hpd[:, 0], ld_hpd[:, 1], color='r', alpha=0.5)
plt.xlabel(x_n[0], fontsize=16)
plt.ylabel(x_n[1], fontsize=16)
plt.savefig('img510.png')
plt.close()

In [29]:
iris = sns.load_dataset("iris")
y_s = pd.Categorical(iris['species']).codes
x_n = iris.columns[:-1]
x_s = iris[x_n].values
x_s = (x_s - x_s.mean(axis=0)) / x_s.std(axis=0)

In [30]:
x_s.shape

(150, 4)

In [31]:
import theano.tensor as tt

with pm.Model() as model_s:
    alpha = pm.Normal('alpha', mu=0, sd=2, shape=3)
    beta = pm.Normal('beta', mu=0, sd=2, shape=(4, 3))
    mu = alpha + pm.math.dot(x_s, beta)
    theta = tt.nnet.softmax(mu)
    yl = pm.Categorical('yl', p=theta, observed=y_s)
    trace_s = pm.sample(2000, njobs=1)

pm.traceplot(trace_s)
plt.savefig('img512.png')
plt.close()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [beta, alpha]
100%|██████████| 2500/2500 [00:13<00:00, 185.95it/s]


In [32]:
pm.gelman_rubin(trace_s)

{'alpha': array([1., 1., 1.]), 'beta': array([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])}

In [33]:
trace_alpha = trace_s['alpha']
trace_alpha.shape

(4000, 3)

In [34]:
trace_beta = trace_s['beta']
trace_beta.shape

(4000, 4, 3)

In [35]:
data_pred = trace_alpha.mean(axis=0) + np.dot(x_s, trace_beta.mean(axis=0))
print('data_pred.shape', data_pred.shape)
y_pred = []
print('data_pred[0].shape', data_pred[0].shape)
for point in data_pred:
    y_pred.append(np.exp(point) / np.sum(np.exp(point), axis=0))
print('len(y_pred)', len(y_pred))
print('y_pred[0].shape', y_pred[0].shape)
np.sum(y_s == np.argmax(y_pred, axis=1)) / len(y_s)

data_pred.shape (150, 3)
data_pred[0].shape (3,)
len(y_pred) 150
y_pred[0].shape (3,)


0.9733333333333334

In [36]:
pm.summary(trace_s)

,mean,sd,mc_error,hpd_2.5,hpd_97.5,n_eff,Rhat
alpha__0,-0.52,1.53,0.03,-3.38,2.54,3715.65,1.0
alpha__1,3.13,1.33,0.02,0.57,5.71,3378.43,1.0
alpha__2,-2.66,1.40,0.02,-5.29,0.29,3264.42,1.0
beta__0_0,-1.51,1.71,0.03,-4.82,1.83,4132.15,1.0
beta__0_1,1.06,1.37,0.02,-1.68,3.56,3750.82,1.0
beta__0_2,0.53,1.38,0.02,-2.15,3.26,3824.89,1.0
beta__1_0,1.82,1.39,0.02,-1.14,4.27,3188.23,1.0
beta__1_1,-0.55,1.24,0.02,-2.97,1.87,2566.94,1.0
beta__1_2,-1.23,1.26,0.02,-3.64,1.27,2843.76,1.0
beta__2_0,-3.21,1.73,0.03,-6.59,0.17,4575.41,1.0


In [37]:
with pm.Model() as model_s:
    alpha = pm.Normal('alpha', mu=0, sd=2, shape=2)
    beta = pm.Normal('beta', mu=0, sd=2, shape=(4, 2))
    alpha_f = tt.concatenate([[0], alpha])
    beta_f = tt.concatenate([np.zeros((4, 1)), beta], axis=1)
    mu = alpha_f + pm.math.dot(x_s, beta_f)
    theta = tt.nnet.softmax(mu)
    yl = pm.Categorical('yl', p=theta, observed=y_s)
    trace_sf = pm.sample(2000, njobs=1)

pm.traceplot(trace_sf)
plt.savefig('img513.png')
plt.close()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [beta, alpha]
100%|██████████| 2500/2500 [00:11<00:00, 215.01it/s]


In [38]:
pm.gelman_rubin(trace_sf)

{'alpha': array([1., 1.]), 'beta': array([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])}

In [39]:
pm.summary(trace_sf)

,mean,sd,mc_error,hpd_2.5,hpd_97.5,n_eff,Rhat
alpha__0,2.61,0.84,0.02,1.09,4.31,2066.69,1.0
alpha__1,-2.41,1.11,0.02,-4.61,-0.30,2201.70,1.0
beta__0_0,2.09,1.11,0.02,-0.19,4.11,1977.38,1.0
beta__0_1,1.60,1.15,0.02,-0.66,3.83,2039.65,1.0
beta__1_0,-1.87,0.70,0.01,-3.33,-0.57,2368.41,1.0
beta__1_1,-2.35,0.83,0.02,-3.99,-0.74,2497.74,1.0
beta__2_0,1.74,1.28,0.03,-0.66,4.32,2208.51,1.0
beta__2_1,5.48,1.46,0.03,2.54,8.30,2520.80,1.0
beta__3_0,1.15,1.25,0.03,-1.20,3.70,2202.82,1.0
beta__3_1,5.68,1.37,0.02,3.06,8.34,2774.04,1.0


In [40]:
trace_alpha = trace_sf['alpha']
zeros = np.zeros((4000, 1))
trace_alpha = np.concatenate([zeros, trace_alpha], axis=1)
print('A', trace_alpha.shape)

trace_beta = trace_sf['beta']
zeros = np.zeros((4000, 4, 1))
trace_beta = np.concatenate([zeros, trace_beta], axis=2)
print('B', trace_beta.shape)

data_pred = trace_alpha.mean(axis=0) + np.dot(x_s, trace_beta.mean(axis=0))
y_pred = []
print('data_pred[0].shape', data_pred[0].shape)
for point in data_pred:
    y_pred.append(np.exp(point) / np.sum(np.exp(point), axis=0))
print('len(y_pred)', len(y_pred))
print('y_pred[0].shape', y_pred[0].shape)
np.sum(y_s == np.argmax(y_pred, axis=1)) / len(y_s)

A (4000, 3)
B (4000, 4, 3)
data_pred[0].shape (3,)
len(y_pred) 150
y_pred[0].shape (3,)


0.9733333333333334

In [41]:
with pm.Model() as model_lda:
    mus = pm.Normal('mus', mu=0, sd=10, shape=2)
    sigma = pm.HalfCauchy('sigma', 5)
    setosa = pm.Normal('setosa', mu=mus[0], sd=sigma, observed=x_0[:50])
    versicolor = pm.Normal('versicolor', mu=mus[1], sd=sigma, observed=x_0[50:])
    bd = pm.Deterministic('bd', (mus[0] + mus[1]) / 2)
    trace_lda = pm.sample(5000, njobs=1)

pm.traceplot(trace_lda)
plt.savefig('img514.png')
plt.close()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [sigma, mus]
100%|██████████| 5500/5500 [00:06<00:00, 839.68it/s] 
The acceptance probability does not match the target. It is 0.8913916849381465, but should be close to 0.8. Try to increase the number of tuning steps.


In [42]:
pm.summary(trace_lda)

,mean,sd,mc_error,hpd_2.5,hpd_97.5,n_eff,Rhat
mus__0,5.01,0.06,6.52e-04,4.88,5.13,10983.51,1.0
mus__1,5.94,0.06,5.98e-04,5.81,6.06,10140.72,1.0
sigma,0.45,0.03,3.11e-04,0.39,0.51,10259.14,1.0
bd,5.47,0.05,4.25e-04,5.38,5.56,10923.74,1.0
